# Analyse Exploratoire des Données - House Prices Prediction
## Laplace Immo - Projet Data Science

Ce notebook présente l'analyse exploratoire complète du dataset House Prices.
Objectif: Comprendre les données et identifier les patterns pour la prédiction des prix.

In [ ]:
# Import des bibliothèques
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Configuration du style
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 10
sns.set_style("whitegrid")

# Chargement des données
train_df = pd.read_csv('../data/raw/train.csv')

## 1. Aperçu des Données

In [ ]:
print("=== APERÇU DES DONNÉES ===")
print(f"Dimensions du dataset: {train_df.shape}")
print(f"Nombre de variables explicatives: {train_df.shape[1] - 2}")  # -2 pour Id et SalePrice
print(f"\nTypes de données:")
print(train_df.dtypes.value_counts())

print(f"\nPremières lignes:")
train_df.head()

## 2. Analyse de la Variable Cible (SalePrice)

In [ ]:
print("=== ANALYSE DE LA VARIABLE CIBLE ===")
target = train_df['SalePrice']

# Statistiques descriptives
target_stats = {
    'Moyenne': target.mean(),
    'Médiane': target.median(),
    'Écart-type': target.std(),
    'Minimum': target.min(),
    'Maximum': target.max(),
    'Q1': target.quantile(0.25),
    'Q3': target.quantile(0.75),
    'Asymétrie': target.skew(),
    'Kurtosis': target.kurtosis()
}

for key, value in target_stats.items():
    print(f"{key}: {value:,.2f}")

# Visualisation de la distribution
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Histogramme
axes[0].hist(target, bins=50, alpha=0.7, color='skyblue', edgecolor='black')
axes[0].axvline(target.mean(), color='red', linestyle='--', label=f'Moyenne: {target.mean():,.0f}')
axes[0].axvline(target.median(), color='green', linestyle='--', label=f'Médiane: {target.median():,.0f}')
axes[0].set_xlabel('Prix de vente ($)')
axes[0].set_ylabel('Fréquence')
axes[0].set_title('Distribution des prix de vente')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Box plot
axes[1].boxplot(target, vert=True)
axes[1].set_ylabel('Prix de vente ($)')
axes[1].set_title('Box plot des prix de vente')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('../reports/target_distribution.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"\nL'asymétrie de {target_stats['Asymétrie']:.2f} indique une distribution légèrement asymétrique vers la droite.")

## 3. Analyse des Valeurs Manquantes

In [ ]:
print("=== ANALYSE DES VALEURS MANQUANTES ===")

# Calcul des valeurs manquantes
missing_values = train_df.isnull().sum()
missing_pct = (missing_values / len(train_df)) * 100

missing_df = pd.DataFrame({
    'Missing Count': missing_values,
    'Missing Percentage': missing_pct
}).sort_values('Missing Percentage', ascending=False)

# Afficher seulement les colonnes avec valeurs manquantes
missing_cols = missing_df[missing_df['Missing Count'] > 0]
print(f"Nombre de colonnes avec valeurs manquantes: {len(missing_cols)}")
print("\nTop 15 des colonnes avec le plus de valeurs manquantes:")
print(missing_cols.head(15))

# Visualisation des valeurs manquantes
plt.figure(figsize=(12, 8))
sns.heatmap(train_df.isnull(), cbar=True, yticklabels=False, cmap='viridis', 
            cbar_kws={'label': 'Valeurs manquantes'})
plt.title('Carte des valeurs manquantes')
plt.tight_layout()
plt.savefig('../reports/missing_values_heatmap.png', dpi=300, bbox_inches='tight')
plt.show()

## 4. Analyse des Corrélations

In [ ]:
print("=== ANALYSE DES CORRÉLATIONS ===")

# Calcul des corrélations avec SalePrice
correlations = train_df.select_dtypes(include=['int64', 'float64']).corr()['SalePrice'].sort_values(ascending=False)

print("Top 15 features les plus corrélées avec SalePrice:")
print(correlations.head(15))
print("\nTop 5 features les moins corrélées (négativement):")
print(correlations.tail(5))

# Visualisation des corrélations
plt.figure(figsize=(12, 10))
top_features = correlations.head(15).index.tolist()
corr_matrix = train_df[top_features].corr()

sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', center=0, 
            square=True, fmt='.2f', cbar_kws={'label': 'Coefficient de corrélation'})
plt.title('Matrice de corrélation des features les plus corrélées avec SalePrice')
plt.tight_layout()
plt.savefig('../reports/correlation_matrix.png', dpi=300, bbox_inches='tight')
plt.show()

## 5. Analyse des Features Clés

In [ ]:
print("=== ANALYSE DES FEATURES CLÉS ===")

# Features les plus corrélées
key_features = ['OverallQual', 'GrLivArea', 'GarageCars', 'TotalBsmtSF', '1stFlrSF']

fig, axes = plt.subplots(2, 3, figsize=(18, 12))
axes = axes.ravel()

for i, feature in enumerate(key_features):
    if i < len(axes):
        axes[i].scatter(train_df[feature], train_df['SalePrice'], alpha=0.5, color='steelblue')
        axes[i].set_xlabel(feature)
        axes[i].set_ylabel('SalePrice')
        axes[i].set_title(f'{feature} vs SalePrice (corr: {train_df[feature].corr(train_df["SalePrice"]):.3f})')
        axes[i].grid(True, alpha=0.3)

# Supprimer le subplot vide
if len(key_features) < len(axes):
    fig.delaxes(axes[-1])

plt.tight_layout()
plt.savefig('../reports/key_features_analysis.png', dpi=300, bbox_inches='tight')
plt.show()

# Analyse de la qualité globale (OverallQual)
print("\nAnalyse de OverallQual:")
qual_analysis = train_df.groupby('OverallQual')['SalePrice'].agg(['mean', 'median', 'count']).round(0)
print(qual_analysis)

## 6. Analyse des Variables Catégorielles

In [ ]:
print("=== ANALYSE DES VARIABLES CATÉGORIELLES ===")

# Sélection des variables catégorielles importantes
categorical_features = ['Neighborhood', 'HouseStyle', 'Exterior1st', 'MasVnrType', 'Foundation']

fig, axes = plt.subplots(2, 3, figsize=(20, 12))
axes = axes.ravel()

for i, feature in enumerate(categorical_features):
    if i < len(axes) and feature in train_df.columns:
        # Box plot pour les variables catégorielles
        sns.boxplot(data=train_df, x=feature, y='SalePrice', ax=axes[i])
        axes[i].set_xticklabels(axes[i].get_xticklabels(), rotation=45)
        axes[i].set_title(f'{feature} vs SalePrice')
        axes[i].grid(True, alpha=0.3)

# Supprimer le subplot vide
if len(categorical_features) < len(axes):
    fig.delaxes(axes[-1])

plt.tight_layout()
plt.savefig('../reports/categorical_features_analysis.png', dpi=300, bbox_inches='tight')
plt.show()

# Analyse du quartier (Neighborhood)
print("\nAnalyse par quartier (top 10):")
neighborhood_analysis = train_df.groupby('Neighborhood')['SalePrice'].agg(['mean', 'median', 'count']).sort_values('mean', ascending=False)
print(neighborhood_analysis.head(10))

## 7. Ingénierie des Features

In [ ]:
print("=== INGÉNIERIE DES FEATURES ===")

# Création de nouvelles features
train_df['HouseAge'] = train_df['YrSold'] - train_df['YearBuilt']
train_df['RemodAge'] = train_df['YrSold'] - train_df['YearRemodAdd']
train_df['TotalSF'] = train_df['GrLivArea'] + train_df['TotalBsmtSF'].fillna(0)
train_df['TotalBathrooms'] = (
    train_df['FullBath'] + 0.5 * train_df['HalfBath'] +
    train_df['BsmtFullBath'].fillna(0) + 0.5 * train_df['BsmtHalfBath'].fillna(0)
)
train_df['OverallScore'] = train_df['OverallQual'] * train_df['OverallCond']

# Analyse des nouvelles features
new_features = ['HouseAge', 'RemodAge', 'TotalSF', 'TotalBathrooms', 'OverallScore']
print("Corrélations des nouvelles features avec SalePrice:")
for feature in new_features:
    corr = train_df[feature].corr(train_df['SalePrice'])
    print(f"{feature}: {corr:.3f}")

# Distribution de l'âge des maisons
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.hist(train_df['HouseAge'], bins=30, alpha=0.7, color='lightcoral', edgecolor='black')
plt.xlabel('Âge de la maison (années)')
plt.ylabel('Fréquence')
plt.title('Distribution de l\\'âge des maisons')
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
plt.scatter(train_df['TotalSF'], train_df['SalePrice'], alpha=0.5, color='seagreen')
plt.xlabel('Surface totale (pi²)')
plt.ylabel('SalePrice')
plt.title(f'TotalSF vs SalePrice (corr: {train_df["TotalSF"].corr(train_df["SalePrice"]):.3f})')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('../reports/feature_engineering.png', dpi=300, bbox_inches='tight')
plt.show()

## 8. Résumé et Insights

In [ ]:
print("=== RÉSUMÉ DE L'ANALYSE EXPLORATOIRE ===")
print("\n1. CARACTÉRISTIQUES DU DATASET:")
print(f"   - {train_df.shape[0]} maisons dans le dataset")
print(f"   - {train_df.shape[1]} variables (dont 79 explicatives)")
print(f"   - 19 variables avec valeurs manquantes")

print("\n2. VARIABLE CIBLE (SalePrice):")
print(f"   - Moyenne: ${train_df['SalePrice'].mean():,.0f}")
print(f"   - Médiane: ${train_df['SalePrice'].median():,.0f}")
print(f"   - Étendue: ${train_df['SalePrice'].min():,.0f} - ${train_df['SalePrice'].max():,.0f}")
print(f"   - Asymétrie positive de {train_df['SalePrice'].skew():.2f}")

print("\n3. FEATURES LES PLUS IMPORTANTES:")
top_features = ['OverallQual', 'GrLivArea', 'TotalSF', 'GarageCars', 'TotalBsmtSF']
for i, feature in enumerate(top_features, 1):
    corr = train_df[feature].corr(train_df['SalePrice'])
    print(f"   {i}. {feature}: r = {corr:.3f}")

print("\n4. INSIGHTS CLÉS:")
print("   - La qualité globale (OverallQual) est le meilleur prédicteur")
print("   - La surface habitable (GrLivArea) est très corrélée au prix")
print("   - L'âge de la maison a un impact négatif sur le prix")
print("   - Les quartiers premium (NoRidge, NridgHt) ont des prix élevés")
print("   - Les garages et sous-sols finis ajoutent de la valeur")
print("   - Les nouvelles features créées (TotalSF, HouseAge) sont pertinentes")

## 9. Préparation pour la Modélisation

In [ ]:
print("=== PRÉPARATION POUR LA MODÉLISATION ===")

# Sauvegarder les données nettoyées
train_df.to_csv('../data/processed/train_cleaned.csv', index=False)
print("Données nettoyées sauvegardées dans: ../data/processed/train_cleaned.csv")

# Créer un résumé des features importantes
feature_importance = pd.DataFrame({
    'Feature': correlations.index[1:21],  # Exclure SalePrice
    'Correlation': correlations.values[1:21]
})

feature_importance.to_csv('../reports/feature_importance.csv', index=False)
print("Importance des features sauvegardée dans: ../reports/feature_importance.csv")

print(f"\nTop 10 features pour la modélisation:")
print(feature_importance.head(10))